# Functional Programming 

Elements of functional programming with Python.

## Resources

[Functional Programming in Python
By Marcus Sanatan](https://stackabuse.com/functional-programming-in-python/)

[Clean Architecture - Uncle Bob Martin](https://www.amazon.co.uk/Clean-Architecture-Craftsmans-Software-Structure/dp/0134494164)

[What is Functional Programming? - Scott Murphy](https://www.youtube.com/watch?v=KHojnWHemO0)

## What is functional programming?

- variables that don't vary (immutable data)
- first class functions (no objects - aka no state)
- no side effects

> Discipline imposed on variable assignment - Uncle Bob Martin

> Isolate transformation of data from definition of the data - Scott Murphy

Examples of functional languages include Lisp, Haskell, Erlang, Clojure

Also a useful idea when thinking about implementations
- [John Carmack on Parallel Implementations](http://sevangelatos.com/john-carmack-on-parallel-implementations/)
- implement new ideas in parallel with the old ones, rather than mutating the existing code
- easy to switch out implementations if the idea is expressed as a pure function
- internal state / multiple entry points = harder

Python has many of the components of a functional language
- map, filter, sum etc

## No side effects

- same inputs -> same outputs (always)
- no dependency on the state of the outside world

Not doing something like:

In [ ]:
def pipeline(data):
    new_data = clean(data)
    database.save(new_data)
    database.load(features)
    return features

## Variables that don't vary

Variables are only initialized 
- they are never changed 
- this avoids problems such as race conditions / deadlocks

Variables being immutable means we can't do:

In [ ]:
#  can't do this - mutates the object in place
def f(x):
    x += 1
    return x

#  can do this - creates a new object
def f(x):
    x = x + 1
    return x

## Functions are first class

First class citizens means we can pass functions around like other variables 
- also known as higher order functions

Below we pass the `sum` function into a generic `reducer` function:

In [1]:
def reducer(func, data):
    return func(data)

data = [1, 2, 3]
reducer(sum, data)

6

Passing in another function gives different results:

In [2]:
reducer(len, data)

3

## Map

Similar to apply in pandas.

Applying a function to each element of an iterable:

In [8]:
def lower(s):
    return s.lower()

cities = ['Berlin', 'Auckland', 'London', 'Sheffield']
m = map(lower, cities)
m

We can see that Python has returned a map object, not the transformed data.  This is an example of **lazy computation**, which is a two step process:
1. build a pipeline/graph

2. put data through it when needed

Examples include Tensorflow 1, Spark, Python generators.

We can get the next element as we would for any kind of generator:

As we are more impatient than lazy, we can get all the processed data by calling `list` on the generator:

In [5]:
list(map(lower, cities))

['berlin', 'auckland', 'london', 'sheffield']

In [6]:
cities

['Berlin', 'Auckland', 'London', 'Sheffield']

## Lambda functions

Anonymous - not assigned to a variable
- possible to have objects with no variable reference (until they get garbage collected :)

We can do the same example using a lambda:

In [9]:
list(map(lambda x: x.lower(), cities))

['berlin', 'auckland', 'london', 'sheffield']

The object we define above is a lambda function:

In [15]:
lw = lambda x: x.lower()

We can do more complex things in lambdas, such as accessing elements of the input data:

In [17]:
populations = [
    ('Berlin', 3.7, 'eu'),
    ('Auckland', 1.7, 'pac'),
    ('London', 8.9, 'eu'),
    ('Sheffield', 0.5, 'eu')
]

list(map(lambda x: (x[0], x[1] * 1000), populations))

[('Berlin', 3700.0),
 ('Auckland', 1700.0),
 ('London', 8900.0),
 ('Sheffield', 500.0)]

Note that we have total flexibility in what data structure we use in the iterable, and how we interact with it in the lambda.

For example, we could map over a sequence of namedtuples, and access elements using the attribute:

In [19]:
from collections import namedtuple

pop = namedtuple('city', ['city', 'population', 'continent'])

populations = [pop(*p) for p in populations]

list(map(lambda x: (x.city, x.population * 1000), populations))

[('Berlin', 3700.0),
 ('Auckland', 1700.0),
 ('London', 8900.0),
 ('Sheffield', 500.0)]

## Reduce

Reduce is an operation that will reduce a sequence to single values (either a single value for the entire sequence, or one single value per group).  Also known as aggregation or a groupby.

Above we defined a function `reducer` to demonstrate that functions are first class citizens in Python.  

The Python standard library also has it's own reduce function - `functools.reduce`.  This function operates on each element and accumulates, returning a single aggregated example.

We can use this `reduce` function on our `populations` dataset
- define a lambda function that adds on the population of the sequence to the total
- our sequence populations
- an initial value of `0`

In [23]:
total = 0
for p in populations:
    total += p.population
total

14.8

In [20]:
from functools import reduce

reduce(lambda total, pop: total + pop[1], populations, 0)

14.8

We can use this `reduce` function to groupby continent:

In [27]:
reduce?

Docstring:
reduce(function, sequence[, initial]) -> value

Apply a function of two arguments cumulatively to the items of a sequence,
from left to right, so as to reduce the sequence to a single value.
For example, reduce(lambda x, y: x+y, [1, 2, 3, 4, 5]) calculates
((((1+2)+3)+4)+5).  If initial is present, it is placed before the items
of the sequence in the calculation, and serves as a default when the
sequence is empty.
Type:      builtin_function_or_method


In [24]:
def gb(acc, city):
    acc[city.continent].append(city.city)
    return acc

reduce(
    gb,
    populations,
    {'eu': [], 'pac': []}
)

{'eu': ['Berlin', 'London', 'Sheffield'], 'pac': ['Auckland']}

## Filter

Similar to boolean masking in pandas/numpy.

Tests each element, keeps those that pass:

In [25]:
list(filter(lambda x: x[1] > 1.0, populations))

[city(city='Berlin', population=3.7, continent='eu'),
 city(city='Auckland', population=1.7, continent='pac'),
 city(city='London', population=8.9, continent='eu')]

## Practical

Create a data processing pipeline that selects the cities that have populations greater than the average of all cities

Two steps:
- `reduce` to find the average of all cities
- `filter` to select using cities above that average

These don't have to be done in a single line!

In [33]:
populations

def avg(total, nxt):
    return (total[0]+nxt[1], total[1]+1)
total = reduce(avg, populations, (0, 0))

# total = (sum, count)
mean = total[0] / total[1]

print(mean)
list(filter(lambda x: x.population > mean, populations))

3.7


[city(city='London', population=8.9, continent='eu')]

You can also do this by incrementally updating the mean:

In [32]:
# state = (mean, num)
def incremental_mean(state, nxt):
    state[1] += 1
    state[0] = state[0] + (nxt.population - state[0]) / state[1]
    return state

reduce(incremental_mean, populations, [0, 0])[0]

3.7000000000000006

## Practical

Implement the same pipeline using two list comprehensions

## Question

In Python, any combination of `map` & `filter` can be done with a single list comprehension
- why do we need to use two in the example above?

## Practical

Create a data processing pipeline that finds the average population for both continents
- step one = reduce to (key, (populations))
- step two = map from (key, (populations)) to (key, avg)

In [29]:
populations

def gb(acc, city):
    acc[city.continent].append(city.population)
    return acc# acc is initialized as a dict here (see below)

pop_cont = reduce(
    gb,
    populations,
    {'eu': [], 'pac': []}
)

list(map(lambda kv: (kv[0], sum(kv[1]) / len(kv[1])), pop_cont.items()))

[('eu', 4.366666666666667), ('pac', 1.7)]